# Άσκηση #2, Κωδικοποίηση Shannon-Fano:
Θεωρήστε τις εικόνες αποχρώσεων του γκρι `airplane.jpg` και `bridge.jpg`.  
 
**Ζητούμενα:**

Α. Υλοποιήστε και εφαρμόστε τη μέθοδο συμπίεσης-κωδικοποίησης Shannon-Fano για τις εικόνες “airplane.jpg” και “bridge.jpg”. Να μη γίνει χρήση έτοιμων συναρτήσεων - υλοποιήσεων της μεθόδου. 

Β. Για κάθε εικόνα παραθέστε: 
1) Τις κωδικές λέξεις που προκύπτουν (αντιστοιχία τιμών φωτεινότητας με δυαδικές κωδικές λέξεις), 
2) Το μέσο μήκος κωδικής λέξης, 
3) Την  τιμή της εντροπίας, και 
4) Το λόγο συμπίεσης που επιτυγχάνεται. 

Γ. Για κάθε εικόνα, υπολογίστε τον αντίστοιχο λόγο συμπίεσης που επιτυγχάνεται με χρήση της μεθόδου συμπίεσης-κωδικοποίησης Huffman. Συγκρίνετε και σχολιάστε τα αποτελέσματα.

In [57]:
%pip install numpy scikit-image huffman --quiet

Note: you may need to restart the kernel to use updated packages.


In [58]:
import numpy as np
import skimage as ski
import huffman
import pandas as pd
import matplotlib.pyplot as plt

def shannon_fano(symbols_probs):
    if len(symbols_probs) == 1:
        return {symbols_probs[0][0]: ''}
    total = sum(prob for _, prob in symbols_probs)
    acc = 0
    split_idx = 0
    for i, (_, prob) in enumerate(symbols_probs):
        acc += prob
        if acc >= total / 2:
            split_idx = i + 1
            break
    left = shannon_fano(symbols_probs[:split_idx])
    right = shannon_fano(symbols_probs[split_idx:])
    for k in left:
        left[k] = '0' + left[k]
    for k in right:
        right[k] = '1' + right[k]
    return {**left, **right}

def image_statistics(image):
    image = image.astype(np.uint8)

    values, counts = np.unique(image.ravel(), return_counts=True)
    probs = counts / counts.sum()

    symbols_probs = sorted(zip(values, probs), key=lambda x: -x[1])
    symbols_counts = list(zip(values, counts))

    return values, probs, counts, symbols_probs, symbols_counts


image_airplane = ski.io.imread("https://github.com/KaratziasK/Digital-Image-Processing/blob/main/project-2/instructions/images-project-2/airplane.jpg?raw=true")
image_bridge = ski.io.imread("https://github.com/KaratziasK/Digital-Image-Processing/blob/main/project-2/instructions/images-project-2/bridge.jpg?raw=true")



# Airplane image
values, probs, counts, symbols_probs, symbols_counts = image_statistics(image_airplane)
shannon_codebook_airplane = shannon_fano(symbols_probs)

encoded_airplane = ''.join([shannon_codebook_airplane[pixel] for pixel in image_airplane.ravel()])

average_length_airplane = sum(probs[i] * len(shannon_codebook_airplane[values[i]]) for i in range(len(values)))

entropy_airplane = -np.sum([p * np.log2(p) for p in probs if p > 0])

# Λόγος συμπίεσης
original_bits = image_airplane.size * 8
compressed_bits = len(encoded_airplane)
compression_ratio_airplane = original_bits / compressed_bits



# bridge image
values, probs, counts, symbols_probs, symbols_counts = image_statistics(image_bridge)
shannon_codebook_bridge = shannon_fano(symbols_probs)

encoded_bridge = ''.join([shannon_codebook_bridge[pixel] for pixel in image_bridge.ravel()])

average_length_bridge = sum(probs[i] * len(shannon_codebook_bridge[values[i]]) for i in range(len(values)))

entropy_bridge = -np.sum([p * np.log2(p) for p in probs if p > 0])

# Λόγος συμπίεσης
original_bits = image_bridge.size * 8
compressed_bits = len(encoded_bridge)
compression_ratio_bridge = original_bits / compressed_bits



# Huffman για airplane
values, probs, counts, symbols_probs, symbols_counts = image_statistics(image_airplane)
huffman_codebook_airplane = huffman.codebook(symbols_probs)
encoded_huffman_airplane = ''.join([huffman_codebook_airplane[pixel] for pixel in image_airplane.ravel()])
average_length_huffman_airplane = sum(probs[i] * len(huffman_codebook_airplane[values[i]]) for i in range(len(values)))
compressed_bits_huffman_airplane = len(encoded_huffman_airplane)
original_bits_airplane = image_airplane.size * 8
compression_ratio_huffman_airplane = original_bits_airplane / compressed_bits_huffman_airplane

# Huffman για bridge
values_b, probs_b, counts_b, symbols_probs_b, symbols_counts_b = image_statistics(image_bridge)
huffman_codebook_bridge = huffman.codebook(symbols_probs_b)
encoded_huffman_bridge = ''.join([huffman_codebook_bridge[pixel] for pixel in image_bridge.ravel()])
average_length_huffman_bridge = sum(probs_b[i] * len(huffman_codebook_bridge[values_b[i]]) for i in range(len(values_b)))
compressed_bits_huffman_bridge = len(encoded_huffman_bridge)
original_bits_bridge = image_bridge.size * 8
compression_ratio_huffman_bridge = original_bits_bridge / compressed_bits_huffman_bridge


all_values_airplane = sorted(set(shannon_codebook_airplane.keys()) | set(huffman_codebook_airplane.keys()))
data_airplane_codes = []
for val in all_values_airplane:
    sf_code = shannon_codebook_airplane[val] if val in shannon_codebook_airplane else None
    huff_code = huffman_codebook_airplane[val] if val in huffman_codebook_airplane else None
    data_airplane_codes.append([val, sf_code, huff_code])
df_codes_airplane = pd.DataFrame(data_airplane_codes, columns=["value", "SF_code", "Huffman_code"])

print("===== airplane.jpg =====")
#pd.reset_option('display.max_rows')
# pd.set_option('display.max_rows', None)
display(df_codes_airplane)
print(f"Shannon-Fano:")
print(f"  Μέσο μήκος κωδικού: {average_length_airplane:.4f} bits")
print(f"  Εντροπία:           {entropy_airplane:.4f} bits")
print(f"  Λόγος συμπίεσης:    {compression_ratio_airplane:.4f}")
print(f"Huffman:")
print(f"  Μέσο μήκος κωδικού: {average_length_huffman_airplane:.4f} bits")
print(f"  Λόγος συμπίεσης:    {compression_ratio_huffman_airplane:.4f}")

all_values_bridge = sorted(set(shannon_codebook_bridge.keys()) | set(huffman_codebook_bridge.keys()))
data_bridge_codes = []
for val in all_values_bridge:
    sf_code = shannon_codebook_bridge[val] if val in shannon_codebook_bridge else None
    huff_code = huffman_codebook_bridge[val] if val in huffman_codebook_bridge else None
    data_bridge_codes.append([val, sf_code, huff_code])
df_codes_bridge = pd.DataFrame(data_bridge_codes, columns=["value", "SF_code", "Huffman_code"])
print("\n===== bridge.jpg =====")
#pd.set_option('display.max_rows', None)
display(df_codes_bridge)
print(f"Shannon-Fano:")
print(f"  Μέσο μήκος κωδικού: {average_length_bridge:.4f} bits")
print(f"  Εντροπία:           {entropy_bridge:.4f} bits")
print(f"  Λόγος συμπίεσης:    {compression_ratio_bridge:.4f}")
print(f"Huffman:")
print(f"  Μέσο μήκος κωδικού: {average_length_huffman_bridge:.4f} bits")
print(f"  Λόγος συμπίεσης:    {compression_ratio_huffman_bridge:.4f}")


===== airplane.jpg =====


,value,SF_code,Huffman_code
0,14,1111111111111101,010111111001011110
1,15,1111111111111110,010111111001011101
2,17,1111111111111100,010111111001011111
3,18,11111111111101,010111111001010
4,19,1111111111101,111001010110011
...,...,...,...
215,230,111111111100,1110010101101
216,231,11111111111100,111001010110010
217,232,11111111111001,01011111100100
218,233,11111111111110,0101111110010110


Shannon-Fano:
  Μέσο μήκος κωδικού: 6.8201 bits
  Εντροπία:           6.7274 bits
  Λόγος συμπίεσης:    1.1730
Huffman:
  Μέσο μήκος κωδικού: 6.7579 bits
  Λόγος συμπίεσης:    1.1838

===== bridge.jpg =====


,value,SF_code,Huffman_code
0,0,11000101,01001010
1,1,1111101101,1010110100
2,2,11111110001,11100000010
3,3,11111111010,01000100000
4,4,11111111110,010111101101
...,...,...,...
251,251,1111111011,01011110111
252,252,111111110001,01000100110
253,253,111111111001,110010010011
254,254,11111111011,111111101001


Shannon-Fano:
  Μέσο μήκος κωδικού: 7.7804 bits
  Εντροπία:           7.7249 bits
  Λόγος συμπίεσης:    1.0282
Huffman:
  Μέσο μήκος κωδικού: 7.7478 bits
  Λόγος συμπίεσης:    1.0325


# Σχολιασμός Αποτελεσμάτων
Huffman και Shannon-Fano δίνουν παρόμοια αποτελέσματα.

Ο Huffman έχει λίγο μικρότερο μέσο μήκος κωδικού και καλύτερο λόγο συμπίεσης, όπως περιμένουμε από τη θεωρία.

Και στις δύο εικόνες, τα αποτελέσματα πλησιάζουν πολύ την εντροπία.

Άρα και οι δύο μέθοδοι είναι αποδοτικές, αλλά ο Huffman δίνει το καλύτερο αποτέλεσμα.